# UTSA CS 3793/5233: Assignment-2

**Mendez - Ramiro - (eyi617)**






## Learning Objectives



*   Game Playing
*   Chess - Board Setup & Rules
*   Adversarial Search
*   AI - Random vs MinMax



## Description

This assignment is focused on **game playing** and creating a proper **AI for chess**. 
In the following sections, you will complete a series of tasks to create a chess game board, rules for each chess piece, a Random AI and a MinMax AI that plays a game of chess for both players (white and black).

The base structure of the code is provided. You are supposed to write code for each of the functions. Comments are provided on what should be done. You **CANNOT** use a complete chess library and change the base code structure completely. However, you **CAN** change the code layout and name/format of the functions.


#Chess Board Setup & Rules

In this section, you will write code to import the necessary libraries and create:

1.   **ChessBoard** - This part will contain code to initialize the board, draw the board, get the board state and move piece.
2.   **ChessRules** - This part will contain code for the chess rules for each piece.



##Import Libraries

The code here will contain only **import** statements. A base list of the required libraries are already imported. You will most likely not need any other libraries, but if needed, add the import statements here. As mentioned before, you can not use any premade chess libraries.

In [ ]:
# your code goes here
import string
import random
import os
import sys
import time
from IPython.display import clear_output
import numpy as np
import copy
import math

In [ ]:
class player():
  def __init__(self, color):
    self.score = 0
    self.color = color

  def update_score(self, value):
    self.score = self.score + value
  
  def get_color(self):
    return self.color

# Chess Piece Object Class

In [ ]:
class chess_piece():
  
  def __init__(self, piece_type, location, value, color):
    self.piece_type = piece_type
    self.location = location
    self.value = value
    self.set_color(color)
    self.set_state(0)

  def set_piece_type(self, type):
    self.piece_type = type

  def set_state(self, state):
    self.state = state

  def set_value(self, value):
    self.value = value

  def set_color(self, color):
    self.color = color

  def set_location(self, location):
    self.location = location
  
  def get_state(self):
    return self.state

  def get_piece_type(self):
    return self.piece_type

  def get_is_active(self):
    return self.is_active
  
  def get_location(self):
    return self.location

  def get_value(self):
    return self.value
  
  def get_color(self):
    return self.color

#Chess Piece Types Extending chess_piece

In [ ]:
class pawn(chess_piece):
  def __init__(self, piece_type, location, value, color):
    super().__init__(piece_type, location, value, color)

  def valid_moves(self, chess_board):
    move_list = list()
    pawn_dirs = np.array([[2, 0], [1, 0], [1, 1], [1, -1]])

    # moves flip if white
    if self.color == 'white': 
      pawn_dirs = pawn_dirs * -1
    
    # if state == 0 then pawn hasnt made its first move and can move two spaces
    if self.state == 0: 
      move = [self.location[0]+ pawn_dirs[0][0] , self.location[1] + pawn_dirs[0][1]]
      # move is within the dimension of the board
      if check_boundary(chess_board, move) and location_unoccupied(chess_board, move):  
        move_list.append(move) 
        self.state = 1

    move = [self.location[0]+pawn_dirs[1][0] , self.location[1]+pawn_dirs[1][1]]
    if check_boundary(chess_board, move) and location_unoccupied(chess_board, move):  
      move_list.append(move)

    move = [self.location[0]+pawn_dirs[2][0] , self.location[1]+pawn_dirs[2][1]]
    if check_boundary(chess_board, move) and not location_unoccupied(chess_board, move):  
      move_list.append(move)
    move = [self.location[0]+pawn_dirs[3][0] , self.location[1]+pawn_dirs[3][1]]
    if check_boundary(chess_board, move) and not location_unoccupied(chess_board, move):  
      move_list.append(move)        
  
    return move_list

class queen(chess_piece):
  def __init__(self, piece_type,location, value, color):  
    super().__init__(piece_type, location, value, color)

  def valid_moves(self, chess_board):
    move_list = []
    from_loc = self.get_location()

    ## SAME as "rook"
    # queen directional moves, all vertical, all diagonal and all horizontal moves
    qd = [[1, 1], [1, -1], [-1, 1], [-1, -1], [1, 0], [-1, 0], [0, 1], [0, -1]]
    for dir in qd:
      to_loc = [from_loc[0] + dir[0], from_loc[1] + dir[1]]
      # increment diagonal
      while 1:
        if not check_boundary(chess_board, to_loc):
          break
        if not location_unoccupied(chess_board, to_loc):
          hit_piece = get_space(chess_board, to_loc)
          if hit_piece.get_color() != self.color: 
            move_list.append(to_loc)
          break
        if is_clear_path(chess_board, from_loc, to_loc, dir):
          move_list.append(to_loc)
        else:
          break
        to_loc = [to_loc[0] + dir[0], to_loc[1] + dir[1]]

    ## SAME as "bishop"
    # diagonal moves: down right, down left, up right, up left
    for dir in qd:
      to_loc = [from_loc[0] + dir[0], from_loc[1] + dir[1]]
      # increment diagonal
      while 1:
        if not check_boundary(chess_board, to_loc):
          break
        if not location_unoccupied(chess_board, to_loc):
          hit_piece = get_space(chess_board, to_loc)
          if hit_piece.get_color() != self.color: 
            move_list.append(to_loc)
          break
        if is_clear_path(chess_board, from_loc, to_loc, dir):
          move_list.append(to_loc)
        else:
          break
        to_loc = [to_loc[0] + dir[0], to_loc[1] + dir[1]]
              
    return move_list

class king(chess_piece):
  def __init__(self, piece_type, location, value, color):
    super().__init__(piece_type, location, value, color)

  def valid_moves(self, chess_board):
    count = 0
    move_list = []
    from_loc = self.get_location()
    # king_dirs = [[1, 1], [1, -1], [-1, 1], [-1, -1]]
    #directions: up one, up-left, up-right, down one, down-right, down-left right one, left one
    king_dirs = [[-1,0], [-1,-1], [-1,1], [1,0], [1,1], [1, -1], [0,1], [0, -1]]
    # king_dirs = [[-1,0], [1,0], [0,1], [0,-1]]
    for dir in king_dirs:
      to_loc = [from_loc[0] + dir[0], from_loc[1] + dir[1]]
      while 1:
       
        if not check_boundary(chess_board, to_loc):
          break
        if not location_unoccupied(chess_board, to_loc):
          hit_piece = get_space(chess_board, to_loc)
          if hit_piece.get_color() != self.color: 
            move_list.append(to_loc)
          break
        if is_clear_path(chess_board, from_loc, to_loc, dir):
            move_list.append(to_loc)
        else:
            break

        # prevents king from moving more than one space in any direction
        if count == 0:
          to_loc = [to_loc[0] + dir[0], to_loc[1] + dir[1]]
          count += 1
        else:
          break
       
    return move_list

class bishop(chess_piece):
  def __init__(self, piece_type, location, value, color):
    super().__init__(piece_type, location, value, color)

  def valid_moves(self, board):
    move_list = []
    from_loc = self.get_location()
    # diagonal moves: down right, down left, up right, up left
    dirs = [[1, 1], [1, -1], [-1, 1], [-1, -1]]
    for dir in dirs:
      to_loc = [from_loc[0] + dir[0], from_loc[1] + dir[1]]
      # increment diagonal
      while 1:
        if not check_boundary(chess_board, to_loc):
          break
        if not location_unoccupied(chess_board, to_loc):
          hit_piece = get_space(chess_board, to_loc)
          if hit_piece.get_color() != self.color: 
            move_list.append(to_loc)
        break
        if is_clear_path(chess_board, from_loc, to_loc, dir):
          move_list.append(to_loc)
        else:
          break
        to_loc = [to_loc[0] + dir[0], to_loc[1] + dir[1]]
        
      return move_list

class rook(chess_piece):
  def __init__(self, piece_type, location, value, color):
    super().__init__(piece_type, location, value, color)

  def valid_moves(self, chess_board):
    move_list = []
   
    from_location = self.get_location()
    # Vertical only moves: up , down   Horizontal only moves: right, left
    dirs = [[1, 0], [-1, 0], [0, 1], [0, -1]]
    for dir in dirs:
      to_location = [from_location[0] + dir[0], from_location[1] + dir[1]]
      # increment vertical or horizontal
      while 1:
        if not check_boundary(chess_board, to_location):
          break

        if not location_unoccupied(chess_board, to_location):
          hit_piece = get_space(chess_board, to_location)
          if hit_piece.get_color() != self.get_color():
            move_list.append(to_location)
          break

        if is_clear_path(chess_board, from_location, to_location, dir):
          move_list.append(to_location)
        else:
          break

        to_location = [to_location[0] + dir[0], to_location[1] + dir[1]]
    return move_list

class knight(chess_piece):
  def __init__(self, piece_type, location, value, color):
    super().__init__(piece_type, location, value, color)

  def valid_moves(self, chess_board):
    move_list = list()
    from_loc = self.get_location()
    knight_dirs = [[1,2], [1,-2], [-1,2], [-1,-2], [2,1], [2,-1], [-2,1], [-2,-1]]

    for dir in knight_dirs:
      to_loc = [from_loc[0] + dir[0], from_loc[1] + dir[1]]
      while 1:
        if not check_boundary(chess_board, to_loc):
          break
        if not location_unoccupied(chess_board, to_loc):
          hit_piece = get_space(chess_board, to_loc)
          if hit_piece.get_color() != self.color: 
              move_list.append(to_loc)
          break
        if is_clear_path(chess_board, from_loc, to_loc, dir):
          move_list.append(to_loc)
        else:
          break
        to_loc = [to_loc[0] + dir[0], to_loc[1] + dir[1]]

    return move_list


##ChessBoard

(10 points)

Fill the code in the code structure provided below for the ChessBoard. The main use of this code block write functions to initialize the board, draw the board, get the board state and move piece. You can add any other functions if needed.


#Chess Board Object Class

In [ ]:
# you can add/change the input parameters for each function 
# you can change the function names and also add more functions if needed
def ChessBoardSetup():
  # initialize and return a chess board - create a 2D 8x8 array that has the value for each cell
  # USE the following characters for the chess pieces - lower-case for BLACK and upper-case for WHITE
  # . for empty board cell
  # p/P for pawn
  # r/R for rook
  # t/T for knight
  # b/B for bishop
  # q/Q for queen
  # k/K for king

  board = [['.' for x in range(8)] for x in range(8)]

  board[0][0] = rook('r', [0,0], 5,'black')
  board[0][1] = knight('t', [0,1], 3,'black')
  board[0][2] = bishop('b', [0,2], 3,'black')
  board[0][3] = queen('q', [0,3], 9,'black')
  board[0][4] = king('k', [0,4], 100,'black')
  board[0][5] = bishop('b', [0,5], 3,'black')
  board[0][6] = knight('t', [0,6], 3,'black')
  board[0][7] = rook('r', [0,7], 5,'black')

  for i in range(8):
    board[1][i] = pawn('p',[1,i], 1, 'black')

  for i in range(8):
    board[6][i] = pawn('P', [6,i], 1, 'white')

  board[7][0] = rook('R', [7,0], 5,'white')
  board[7][1] = knight('T', [7,1], 3,'white')
  board[7][2] = bishop('B', [7,2], 3,'white')
  board[7][3] = queen('Q', [7,3], 9,'white')
  board[7][4] = king('K', [7,4], 100,'white')
  board[7][5] = bishop('B', [7,5], 3,'white')
  board[7][6] = knight('T', [7,6], 3,'white')
  board[7][7] = rook('R', [7,7], 5,'white')

  return board

def check_boundary(board, to_loc):
    (x,y) = to_loc
    print(x)
    print(y)
    if (x > 7 or x < 0) or (y > 7 or y < 0):
      return False
    else: 
      return True

def location_unoccupied(board, to_loc):
  if board[to_loc[0]][to_loc[1]] == '.':
    return True
  else: 
    return False

def get_space(chess_board, location):
  return chess_board[location[0], location[1]]

def get_space(chess_board, location):
  return chess_board[location[0]][location[1]]

def get_valid_pieces(chess_board):
  pieces = []
  for c in chess_board:
    for r in c:
      if r != '.':
        pieces.append(r)
  
  return pieces

def DrawBoard(chess_board):
  # write code to print the board - following is one print example
  # r t b q k b t r
  # p p p p p p p p
  # . . . . . . . .
  # . . . . . . . .
  # . . . . . . . .
  # . . . . . . . .
  # P P P P P P P P
  # R T B Q K B T R

  for r in chess_board:
    for c in r:
      if c == '.':  
        print('.', end = " ")
      elif c == 'g':
        print('g', end = " ")
      else:
        print(c.get_piece_type(), end = " ")
    print()

def move_piece(piece, to_loc, chess_board):
  # write code to move the one chess piece
  # you do not have to worry about the validity of the move - this will be done before calling this function
  # this function will at least take the move (from-piece and to-piece) as input and return the new board layout
  from_loc = piece.get_location()

  if chess_board[to_loc[0]][to_loc[1]] == '.':
    chess_board[from_loc[0]][from_loc[1]] = '.'
    piece.set_location(to_loc)
    chess_board[to_loc[0]][to_loc[1]] = piece
      
    return chess_board
  else: 
    print("Position is not empty. Try Again")     

  


In [ ]:
chess_board = ChessBoardSetup()
DrawBoard(chess_board)

# # for moves in piece.valid_moves(chess_board):
# #   print(moves)



r t b q k b t r 
p p p p p p p p 
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 
P P P P P P P P 
R T B Q K B T R 


#Testers for Chess Board & Piece Functions


In [ ]:
piece = chess_board[0][4]
# DrawBoard(chess_board)

# test for piece.valid_moves() before move
# for move in piece.valid_moves(chess_board):
#   print(move)

# # # test for move_piece()
move_piece(piece, [4,4], chess_board)

# # # test for piece.valid_moves after move
for move in piece.valid_moves(chess_board):
  chess_board[move[0]][move[1]] = 'g'
  print(move)

# # # test for get_valid_pieces()
# # # for pieces in get_valid_pieces(chess_board):
# # #   print(pieces)

# # # test for if square location is in chess_board boundary
# # # if check_boundary(chess_board, [1,7]):
# # #   print("True")
# # # else: 
# # #   print("false")

# # # test for if location_unoccupied
# # # if location_unoccupied(chess_board, [2,0]):
# # #   print("True")
# # # else: 
# # #   print("false")

print()
DrawBoard(chess_board)

3
4
2
4
3
4
3
3
3
5
5
4
5
5
5
3
4
5
4
3
[3, 4]
[2, 4]
[3, 3]
[3, 5]
[5, 4]
[5, 5]
[5, 3]
[4, 5]
[4, 3]

r t b q . b t r 
p p p p p p p p 
. . . . g . . . 
. . . g g g . . 
. . . g k g . . 
. . . g g g . . 
P P P P P P P P 
R T B Q K B T R 


##ChessRules

(50 points)

Fill the code in the code structure provided below for ChessRules. The main use of the code block is to write functions to design the rules for movement of each piece on the board. This block will also contain the function to check if the current player is in check, check-mate. You can also have functions that can return the current player's pieces that have legal moves in the current board state. 

Following are some **suggested** functions with the pseudocode provided. You can create/remove functions as needed.


In [ ]:
# return True if the input move (from-square and to-square) is legal, else False
# this is the KEY function which contains the rules for each piece type 
def is_move_legal(piece, to_square, chess_board):
    # input is from-square and to-square
    # use the input and the board to get the from-piece and to-piece

    # if from-square is the same as to-square
        # return False
    if piece.get_location() == to_square:
      return False
    
    moves = piece.valid_moves(chess_board)

    for move in moves:
      if to_square == move:
        return True
      else:
        # return False - if none of the other True's are hit above
        return False

# # gets a list of all pieces for the current player that have legal moves
def get_pieces_with_legal_moves(player, chess_board):
  valid = []
  pieces = get_valid_pieces(chess_board)
  for piece in pieces:
    if piece.color == player:
      if piece.valid_moves != None:
        valid.append(piece)

  return valid

# # returns True if the current player is in checkmate, else False
def is_checkmate(player, chess_board):

  pieces = get_pieces_with_legal_moves(player, chess_board)
  if pieces == None:
    return True
  else: 
    return False

# # returns True if the given player is in Check state
def is_in_check(player, chess_board):
  #gets player pieces and all pieces currently on the board
  player_pieces = get_pieces_with_legal_moves(player, chess_board)
  all_pieces = get_valid_pieces(chess_board)
  # gets current players king
  for piece in player_pieces:
    if piece.get_piece_type().lower() == 'k':
      king = piece
      
  # loop through all valid pieces on board
  for p in all_pieces:
    # if it is opposing player
    if p.get_color != player:
      # if that opponents piece has a move that can take the king
      if is_move_legal(p, king.get_location(), chess_board):
        return True
      else:
        return False


def is_clear_path(chess_board, from_location, to_location, dir):
  tmp_loc = [from_location[0] + dir[0], from_location[1] + dir[1]]
  while tmp_loc != to_location:
    # if true then ran into a piece or out of bounds
    if not check_boundary(chess_board, tmp_loc) or not location_unoccupied(chess_board, tmp_loc):
        return False
    tmp_loc = [tmp_loc[0] + dir[0], tmp_loc[1] + dir[1]]
  return True


# # makes a hypothetical move (from-square and to-square)
# # returns True if it puts current player into check
def does_move_put_player_in_check(from_square, to_square, chess_board):
  from_piece = chess_board[from_square[0]][from_square[1]]
  # temporarily move piece on chess_board
  move_piece(from_piece, to_square, chess_board)
  
  # chess_piece object for temporary move
  to_piece = chess_board[to_square[0]][to_square[1]]
  # save value if the move will put the player in check
  if is_in_check(to_piece.get_color(), chess_board):
    value = True
  else:
    value = False
  # undo temporary move
  move_piece(to_piece, from_square, chess_board)

  return value


In [ ]:
# # pieces = get_pieces_with_legal_moves('black', chess_board)

# # for piece in pieces:
# #   print(f"{piece.get_piece_type()} {piece.get_color()}")

# does_move_put_player_in_check([1,0],[2,0], chess_board)

#Artificial Intelligence

In this section, you will write code for the Artificial Intelligence (AI) that will play a game of chess. You will write 2 types of AI:

1.   **RandomAI** - This part will contain code for moving a chess piece randomly.
2.   **MinMaxAI** - This part will contain code for moving a chess piece using the MinMax strategy discussed in the lecture.


##RandomAI

(10 points)

Complete the function below that will perform a random move for the given player. The function will return the move (from-piece and to-piece). You will most likely not need to write any other function, but you can, if needed.


In [ ]:
def GetRandomMove(player, chess_board):
  # pick a random piece and a random legal move for that piece
  pieces = get_pieces_with_legal_moves(player, chess_board)
  all_moves = []

  for piece in pieces:
    rand_move = piece.valid_moves(chess_board)
    for move in rand_move:
      if is_move_legal(piece, move, chess_board):
        all_moves.append((piece, move))

  return random.choice(all_moves)

##MinMaxAI

(50 points)

Complete the functions below that will perform a move for the given player using the MinMax AI strategy. One function will evaluate the board if a move is performed - give score for each of piece and calculate the score for the entire chess board. In the second function you will write actual code for the MinMax strategy and return the move (from-piece and to-piece). To get the allocated points, searching should be **2-ply (one Max and one Min)**. You will most likely not need to write any other function, but you can, if needed.

## Extra Credit

*   **(5 points)** Modify the above MinMax strategy to be **4-ply (one Max, one Min, one Max, one Min)**.
*   **(10 points)** Perform **alpha-beta pruning** for the MinMax strategy.

# def evl():
    # this function will calculate the score on the board, if a move is performed
    # give score for each of piece and calculate the score for the chess board


# def GetMinMaxMove():
    # return the best move for the current player using the MinMax strategy
    # to get the allocated points, searching should be 2-ply (one Max and one Min)

    # Following is the setup for a 2-ply game

    # pieces = GetPiecesWithLegalMoves(curPlayer)

    # for each piece in pieces
        # moves = GetListOfLegalMoves(curPlayer, piece)
        # for move in moves
            # perform the move temporarily
            # enemyPieces = GetPiecesWithLegalMoves(enemyPlayer)
            # for enemyPiece in penemyPiecesieces
                # enemyMoves = GetListOfLegalMoves(enemyPlayer, enemyPiece)
                # for enemyMove in enemyMoves
                    # perform the enemyMove temporarily
                    # res = evl(curPlayer)
                    # update the bestEnemyMove -- this is the MIN player trying to minimize from the 'res' evaluation values
                    # undo the enemyMove
            # update the bestMove -- this is the MAX player trying to maximize from the 'bestEnemyMove' evaluation values
            # undo the move
    # if bestMove found without any doubt, pick that
    # if bestMove not found, pick randomly

    # OPTIONAL -- sometimes automated chess keeps on performing the moves again and again
    # e.g., move king left one square and then move king back - repeat
    # For this you will need to remember the previous move and see if the current best move is not the same and opposite as the previous move
    # If so, pick the second best move instead of the best move


In [ ]:

# function to get max move for black player
def get_max_move(player, board, depth, alpha, beta):
    # gets list of pieces with legal moves for specific player
    pieces = get_pieces_with_legal_moves(player, board)
    # Player has no moves left
    if not pieces:
        return player, 'There are no more moves.', -1
    max_move = tuple()
    max_moves = []
    max_score = -10000000
    # loop through all valid pieces
    for piece in pieces:
        moves = piece.valid_moves(board)
        # loop through all moves for specific piece
        for move in moves:
          # temporary variables so the main board doesnt get affected
            tmp_piece = copy.deepcopy(piece)
            tmp_board = copy.deepcopy(board)
            tmp_board.move_piece(tmp_piece, move)
            # depth has to be at least one
            if depth != 0:
                (_, min_score, _) = get_min_move(
                    'white', tmp_board, depth-1, alpha, beta)
                # if there is no min moves
                if min_score == 'There are no more moves.':
                    tmp_score = tmp_board.evaluate_board()
                    if tmp_score > max_score:
                        max_moves = []
                        max_score = tmp_score
                        max_move = (piece, max_score, move)
                        max_moves.append(max_move)
                    elif tmp_score == max_score:
                        max_move = (piece, max_score, move)
                        max_moves.append(max_move)
                # if there is a min move available
                else:
                    if min_score > max_score:
                        max_moves = []
                        max_score = min_score
                        max_move = (piece, max_score, move)
                        max_moves.append(max_move)
                    elif min_score == max_score:
                        max_move = (piece, max_score, move)
                        max_moves.append(max_move)
                    if max_score >= beta:
                        return max_move
                    if max_score > alpha:
                        alpha = max_score
            # if depth is 0
            else:
                tmp_score = tmp_board.evaluate_board()
                if tmp_score > max_score:
                    max_moves = []
                    max_score = tmp_score
                    max_move = (piece, max_score, move)
                    max_moves.append(max_move)
                elif tmp_score == max_score:
                    max_move = (piece, max_score, move)
                    max_moves.append(max_move)
    if len(max_moves) > 1:
        return random.choice(max_moves)
    return max_move

# function to get min move for white player
def get_min_move(player, board, depth, alpha, beta):
    pieces = get_pieces_with_legal_moves(player, board)
    # player has no moves left
    if not pieces:
        return player, 'There are no more moves.', 1
    min_move = tuple()
    min_moves = []
    min_score = 10000000
    # loop through all valid pieces
    for piece in pieces:
        moves = piece.valid_moves(board)
        # loop through all moves for specific piece
        for move in moves:
          # temporary variables so the main board doesnt get affected
            tmp_piece = copy.deepcopy(piece)
            tmp_board = copy.deepcopy(board)
            move_piece(tmp_piece, move, tmp_board)
            # if depth is at least equal to 1
            if depth != 0:
                (_, max_score, _) = get_max_move(
                    'black', tmp_board, depth-1, alpha, beta)
                # there are no more max moves left
                if max_score == 'There are no more moves.':
                    tmp_score = evaluate_board(chess_board)
                    if tmp_score < min_score:
                        min_moves = []
                        min_score = tmp_score
                        min_move = (piece, min_score, move)
                        min_moves.append(min_move)
                    elif tmp_score == min_score:
                        min_move = (piece, min_score, move)
                        min_moves.append(min_move)
                # there are max moves that exist
                else:
                    if max_score < min_score:
                        min_moves = list()
                        min_score = max_score
                        min_move = (piece, min_score, move)
                        min_moves.append(min_move)
                    elif max_score == min_score:
                        min_move = (piece, min_score, move)
                        min_moves.append(min_move)
                    if min_score <= alpha:
                        return min_move
                    if min_score < beta:
                        beta = min_score
            # if depth is equal to 1
            else:
                tmp_score = evaluate_board(chess_board)
                if tmp_score < min_score:
                    min_moves = list()
                    min_score = tmp_score
                    min_move = (piece, min_score, move)
                    min_moves.append(min_move)
                elif tmp_score == min_score:
                    min_move = (piece, min_score, move)
                    min_moves.append(min_move)
    if len(min_moves) > 1:
        return random.choice(min_moves)
    return min_move

# gets the best move for the specific players turn
def get_best_move(player, board):
    # if 'black' player than it is assigned the max moves
    if player == 'black':
        (piece, _, move) = get_max_move(player, board, 2, -math.inf, math.inf)
    # if 'white' player than it is assigned the min moves
    else:
        (piece, _, move) = get_min_move(player, board, 2, -math.inf, math.inf)
    return (piece, move)

#Game Setup & Main Loop

(05 points)

Write code below to have a game-play between two AIs - Random vs MinMax. For each iteration, draw the board before and after a turn. 

In [ ]:
# initialize and setup the board
# player assignment and counter initializations
chess_board = ChessBoardSetup()

player1 = player('black')
player2 = player('white')

turns = 0
N = 1000

curr_player = player1

# main game loop - while a player is not in checkmate or stalemate (<N turns)
# below is the rough looping strategy
while turns < N:
  # if current player isnt in checkmate
  if is_checkmate(curr_player.get_color(), chess_board) == False:
    clear_output()
    DrawBoard(chess_board)
    print(f"{curr_player.get_color()} Turn")
    
    # randomAI algorithm
    (piece, move) = GetRandomMove(curr_player.get_color(), chess_board)
    print(f"{piece.get_piece_type()} and {move}")
    
    if  not does_move_put_player_in_check(piece.get_location(), move, chess_board):
      move_piece(piece, move, chess_board)
    
    # minmax algorithm
    # if  not does_move_put_player_in_check(piece.get_location(), move, chess_board):
    #   (best_piece, _, best_move) = get_best_move(curr_player.get_color(), chess_board)
    #   move_piece(best_piece, best_move, chess_board)

    print(f"Number of moves: {turns +1}")
    # write code to take turns and move the pieces
    DrawBoard(chess_board)
    time.sleep(0.5)
    
  else:
    print(f"{curr_player.get_color()} Wins!!")
    
  # change players turn
  if curr_player == player1:
    curr_player = player2
  else:
    curr_player = player1
  turns += 1


  
# check and print - Stalemate or Checkmate


r t b q . b . r 
. p . . p k p p 
p . p p . . . t 
. . . . . p . . 
P . . . . . . . 
. . . . P . . . 
. P P P Q P P P 
R T B K . B T R 
black Turn
1
0
2
0
-1
0
0
1
0
-1
1
0
2
0
-1
0
0
1
0
-1
1
3
2
5
1
3
3
7
1
3
2
5
4
9
1
-1
-1
3
-1
-1
2
2
2
0
-2
2
-2
0
1
3
2
5
1
3
3
7
1
3
2
5
4
9
1
-1
-1
3
-1
-1
2
2
2
0
-2
2
-2
0
1
3
2
5
1
3
3
7
1
3
2
5
4
9
1
-1
-1
3
-1
-1
2
2
2
0
-2
2
-2
0
1
3
2
5
1
3
3
7
1
3
2
5
4
9
1
-1
-1
3
-1
-1
2
2
2
0
-2
2
-2
0
1
3
1
4
1
2
2
1
1
2
3
0
1
2
2
1
4
-1
-1
4
-1
2
1
3
2
3
-1
3
0
4
0
5
0
2
1
4
1
2
2
1
1
2
3
0
1
2
2
1
4
-1
-1
4
-1
2
1
3
2
3
-1
3
0
4
0
5
0
2
1
4
1
2
2
1
1
2
3
0
1
2
2
1
4
-1
-1
4
-1
2
1
3
2
3
-1
3
0
4
0
5
0
2
1
4
1
2
2
1
1
2
3
0
1
2
2
1
4
-1
-1
4
-1
2
1
3
2
3
-1
3
0
4
0
5
0
2
1
4
1
2
2
1
1
2
3
0
1
2
2
1
4
-1
-1
4
-1
2
1
3
2
3
-1
3
0
4
0
5
0
2
1
4
1
2
2
1
1
2
3
0
1
2
2
1
4
-1
-1
4
-1
2
1
3
2
3
-1
3
0
4
0
5
0
2
1
4
1
2
2
1
1
2
3
0
1
2
2
1
4
-1
-1
4
-1
2
1
3
2
3
-1
3
0
4
0
5
0
2
1
4
1
2
2
1
1
2
3
0
1
2
2
1
4
-1
-1
4
-1
2
1
3
2
3
-1
3
0
4
0
5
0
2
1
4
1
2
2
1
1

KeyboardInterrupt: ignored

#Submission Instructions

1.   Complete all tasks above - **File MUST contain the output for ALL cells**
2.   Export this notebook as .ipynb
      (File > Download as ipynb)
3.   Upload the .ipynb file on Blackboard



##Rubric

*   (10 points) Chess Board Setup
*   (50 points) Chess Rules Setup
*   (10 points) Random AI
*   (50 points) MinMax AI (2-ply)
*   (05 points) Game Main Loop - Random vs MinMax
*   (15 points) Extra Credit - 4-ply MinMax + alpha-beta pruning





In [ ]:
class board():

  def __init__(self):
    self.chess_board = np.array()
    self.fill_board()

  def get_piece(self, location):
    return chess_board[location[0]][location[1]]

  def check_boundary(self, to_loc):
    (x,y) = to_loc
    print(x)
    print(y)
    if (x > 7 or x < 0) or (y > 7 or y < 0):
      return False
    else: 
      return True

  def fill_board(self):
  
    # def __init__(self, piece_type, location, value, color):
    self.chess_board[0][0] = rook('r', [0,0], 5,'black')
    self.chess_board[0][1] = knight('t', [0,1], 3,'black')
    self.chess_board[0][2] = bishop('b', [0,2], 3,'black')
    self.chess_board[0][3] = queen('q', [0,3], 9,'black')
    self.chess_board[0][4] = king('k', [0,4], 100,'black')
    self.chess_board[0][5] = bishop('b', [0,5], 3,'black')
    self.chess_board[0][6] = knight('t', [0,6], 3,'black')
    self.chess_board[0][7] = rook('r', [0,7], 5,'black')

    for i in range(8):
      self.chess_board[1][i] = pawn('p',[1,i], 1, 'black')

    for i in range(8):
      self.chess_board[6][i] = pawn('P', [6,i], 1, 'white')

    self.chess_board[7][0] = rook('R', [7,0], 5,'white')
    self.chess_board[7][1] = knight('T', [7,1], 3,'white')
    self.chess_board[7][2] = bishop('B', [7,2], 3,'white')
    self.chess_board[7][3] = queen('Q', [7,3], 9,'white')
    self.chess_board[7][4] = knight('K', [7,4], 100,'white')
    self.chess_board[7][5] = bishop('B', [7,5], 3,'white')
    self.chess_board[7][6] = knight('T', [7,6], 3,'white')
    self.chess_board[7][7] = rook('R', [7,7], 5,'white')


  def draw_board(self):
    # write code to print the board - following is one print example
    # r t b q k b t r
    # p p p p p p p p
    # . . . . . . . .
    # . . . . . . . .
    # . . . . . . . .
    # . . . . . . . .
    # P P P P P P P P
    # R T B Q K B T R
    for r in self.chess_board:
      for c in r:
        if c == '.':
          print('.', end = " ")
        else:
          print(c.get_piece_type(), end = " ")
      print()
   
  def move_piece(self, piece, to_loc):
    # write code to move the one chess piece
    # you do not have to worry about the validity of the move - this will be done before calling this function
    # this function will at least take the move (from-piece and to-piece) as input and return the new board layout
    from_loc = piece.get_location()

    if self.chess_board[to_loc[0]][to_loc[1]] == '.':
      self.chess_board[from_loc[0]][from_loc[1]] = '.'
      piece.set_location(to_loc)
      self.chess_board[to_loc[0]][to_loc[1]] = piece
    else: 
      print("Position is already taken.")


In [ ]:
cboard = board()
cboard.draw_board()



# if cboard.check_boundary([8,7]):
#   print("True")
# else: 
#   print("false")

In [ ]:
piece = cboard.get_piece([2,0])
print(piece)
cboard.move_piece(piece, [1,0])
print()
cboard.draw_board()